# Train the model

In [3]:
import pickle
import numpy as np
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

## Reload the data

In [2]:
# Reload the data

def reload_data(pickle_file):
    print('reload: ', pickle_file)
    with open(pickle_file, 'rb') as f:
        pickle_data = pickle.load(f)
        X_train = pickle_data['X_train']
        y_train = pickle_data['y_train']
        del pickle_data  # Free up memory
    print('X_train shape: ', X_train.shape, 'y_train shape: ',y_train.shape)
    return X_train, y_train

X_train1, y_train1 = reload_data('./pre-data/1.pickle')
X_train2, y_train2 = reload_data('./pre-data/2.pickle')
X_train3, y_train3 = reload_data('./pre-data/3.pickle')
X_train4, y_train4 = reload_data('./pre-data/4.pickle')

reload:  ./pre-data/1.pickle
X_train shape:  (4018, 160, 320, 3) y_train shape:  (4018,)
reload:  ./pre-data/2.pickle
X_train shape:  (4018, 160, 320, 3) y_train shape:  (4018,)
reload:  ./pre-data/3.pickle
X_train shape:  (4018, 160, 320, 3) y_train shape:  (4018,)
reload:  ./pre-data/4.pickle
X_train shape:  (4018, 160, 320, 3) y_train shape:  (4018,)


In [5]:
# Combine the data set
print('combine data set 1, 2, 3, 4 ...')
X_train = np.concatenate((X_train1, X_train2, X_train3, X_train4))
y_train = np.concatenate((y_train1, y_train2, y_train3, y_train4))
X_train, y_train = shuffle(X_train, y_train)
print('X_train shape: ', X_train.shape, 'y_train shape: ',y_train.shape)

combine data set 1, 2, 3, 4 ...
X_train shape:  (16072, 160, 320, 3) y_train shape:  (16072,)


## Model architecture

### model 1: LeNet

In [4]:
lenet = Sequential()
lenet.add(Lambda(lambda x: x/255. - 0.5, input_shape=(160, 320, 3)))
lenet.add(Cropping2D(cropping=((70, 25), (0, 0))))
lenet.add(Convolution2D(6, 5, 5, activation='relu'))
lenet.add(MaxPooling2D())
lenet.add(Convolution2D(6, 5, 5, activation='relu'))
lenet.add(MaxPooling2D())
lenet.add(Flatten())
lenet.add(Dense(120))
lenet.add(Dense(84))
lenet.add(Dense(1))



### model 2: Nvidia

In [13]:
nvidia = Sequential()
nvidia.add(Lambda(lambda x: x/255. - 0.5, input_shape=(160, 320, 3)))
nvidia.add(Cropping2D(cropping=((70, 25), (0, 0))))
nvidia.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
nvidia.add(MaxPooling2D())
nvidia.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
nvidia.add(MaxPooling2D())
# Negative dimension size caused by subtracting 5 from 3 for 
#  'Conv2D_21' (op: 'Conv2D') with input shapes: [?,3,19,36], [5,5,36,48].
# nvidia.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
# nvidia.add(MaxPooling2D())
# nvidia.add(Convolution2D(64, 3, 3, activation='relu'))
# nvidia.add(MaxPooling2D())
# nvidia.add(Convolution2D(64, 3, 3, activation='relu'))
# nvidia.add(MaxPooling2D())
nvidia.add(Flatten())
nvidia.add(Dense(100))
nvidia.add(Dense(50))
nvidia.add(Dense(10))
nvidia.add(Dense(1))

ValueError: Negative dimension size caused by subtracting 5 from 3 for 'Conv2D_21' (op: 'Conv2D') with input shapes: [?,3,19,36], [5,5,36,48].

## Training method

In [6]:
nvidia.compile(loss='mse', optimizer='adam')
nvidia.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=3)
nvidia.save('model.h5')

Train on 12857 samples, validate on 3215 samples
Epoch 1/3
12857/12857 [==============================] - 205s - loss: 0.0810 - val_loss: 0.0128
Epoch 2/3
12857/12857 [==============================] - 45s - loss: 0.0113 - val_loss: 0.0120
Epoch 3/3
12857/12857 [==============================] - 45s - loss: 0.0105 - val_loss: 0.0120
